In [3]:
from utils.branch_bound import matrix_table, branch_bound_tree

In [3]:
from collections import defaultdict

In [4]:
import numpy as np

In [5]:
s = '11101010101'
array_of_integers = np.array(list(s), dtype=int)
print(array_of_integers)

[1 1 1 0 1 0 1 0 1 0 1]


In [5]:
integers = [0, 1, 2, 3, 6, 7, 8, 9, 14, 15]
max_int = max(integers)
print(max_int)
max_bit = format(max_int, 'b')
print(max_bit)
print(len(max_bit))
bit_format = str(len(max_bit))+'b'
print(bit_format)
# Convert integers to binary representations
binary_representations = {integer: format(integer, bit_format).replace(' ', '0') for integer in integers}
print(binary_representations)


15
1111
4
4b
{0: '0000', 1: '0001', 2: '0010', 3: '0011', 6: '0110', 7: '0111', 8: '1000', 9: '1001', 14: '1110', 15: '1111'}


In [6]:
from collections import defaultdict

In [7]:
binary_lists = defaultdict(list)
for integer, bits_str in binary_representations.items():
    bits = []
    for byte in bits_str:
        binary_lists[integer].append(int(byte))
print(binary_lists)

defaultdict(<class 'list'>, {0: [0, 0, 0, 0], 1: [0, 0, 0, 1], 2: [0, 0, 1, 0], 3: [0, 0, 1, 1], 6: [0, 1, 1, 0], 7: [0, 1, 1, 1], 8: [1, 0, 0, 0], 9: [1, 0, 0, 1], 14: [1, 1, 1, 0], 15: [1, 1, 1, 1]})


In [8]:
minterm_table = defaultdict(str)
group_table = defaultdict(list)
for minterm, bits in binary_lists.items():
    uninverted_bits_count = sum(bits)
    
    group_table[uninverted_bits_count].append((minterm,))
    for byte in bits:
        minterm_table[(minterm,)]+=str(byte)

In [9]:
group_table

defaultdict(list,
            {0: [(0,)],
             1: [(1,), (2,), (8,)],
             2: [(3,), (6,), (9,)],
             3: [(7,), (14,)],
             4: [(15,)]})

In [10]:
minterm_table

defaultdict(str,
            {(0,): '0000',
             (1,): '0001',
             (2,): '0010',
             (3,): '0011',
             (6,): '0110',
             (7,): '0111',
             (8,): '1000',
             (9,): '1001',
             (14,): '1110',
             (15,): '1111'})

In [11]:
uninv_bits_counts = list(group_table.keys())
for idx in range(len(uninv_bits_counts)-1):
    current_group, next_group = uninv_bits_counts[idx], uninv_bits_counts[idx+1]
    print(current_group, next_group)

0 1
1 2
2 3
3 4


In [12]:
print(group_table)

defaultdict(<class 'list'>, {0: [(0,)], 1: [(1,), (2,), (8,)], 2: [(3,), (6,), (9,)], 3: [(7,), (14,)], 4: [(15,)]})


In [13]:
print(minterm_table)

defaultdict(<class 'str'>, {(0,): '0000', (1,): '0001', (2,): '0010', (3,): '0011', (6,): '0110', (7,): '0111', (8,): '1000', (9,): '1001', (14,): '1110', (15,): '1111'})


In [14]:
group_table

defaultdict(list,
            {0: [(0,)],
             1: [(1,), (2,), (8,)],
             2: [(3,), (6,), (9,)],
             3: [(7,), (14,)],
             4: [(15,)]})

In [15]:
group_table[1]

[(1,), (2,), (8,)]

In [16]:
group_table[2]

[(3,), (6,), (9,)]

In [17]:
# uninv_bits_counts = list(group_table.keys())
# next_group_table = defaultdict(list)
# absorbed_groups = {}
# for idx in range(len(uninv_bits_counts)-1):
#     current_group, next_group = uninv_bits_counts[idx], uninv_bits_counts[idx+1]
#     for current_minterms in group_table[current_group]:
#         for next_minterms in group_table[next_group]:
#             count = 0
#             variable = ''
#             current_bits = minterm_table[current_minterms]
#             next_bits = minterm_table[next_minterms]
#             for byte_idx in range(0,4):
#                 current_byte = current_bits[byte_idx]
#                 next_byte = next_bits[byte_idx]
#                 if current_byte == next_byte:
#                     if current_byte and next_byte:
#                         variable += '1'
#                     elif (not current_byte) and (not next_byte):
#                         variable += '0'
#                 # elif current_byte != next_byte: 
#                 else:
#                     variable += '-'
#                     count += 1
#             if count >= 3:
#                 continue
#             next_group_table[idx].append((current_minterms, next_minterms))
#             absorbed_groups[(current_minterms, next_minterms)] = variable
            
# print(absorbed_groups)


In [18]:
def simplify_prime_implicants(group_table, minterm_table):
    uninv_bits_counts = list(group_table.keys())
    next_group_table = defaultdict(list)
    next_minterm_table = {}
    unique_bits = set()
    for idx in range(len(uninv_bits_counts)-1):
        current_group, next_group = uninv_bits_counts[idx], uninv_bits_counts[idx+1]
        for current_minterms in group_table[current_group]:
            for next_minterms in group_table[next_group]:
                count = 0
                variable = ''
                current_bits = minterm_table[current_minterms]
                next_bits = minterm_table[next_minterms]
                for byte_idx in range(0,4):
                    current_byte = current_bits[byte_idx]
                    next_byte = next_bits[byte_idx]
                    if current_byte == next_byte:
                        if current_byte=='1' and next_byte=='1':
                            variable += '1'
                        elif current_byte=='0' and next_byte=='0':
                            variable += '0'
                        elif current_byte=='-' and next_byte=='-':
                            variable += '-'
                    else:
                        variable += '-'
                        count += 1
                if count >= 2 or variable in unique_bits:
                    continue
                next_group_table[idx].append((current_minterms+next_minterms))
                next_minterm_table[current_minterms+next_minterms] = variable
                unique_bits.add(variable)
    return next_group_table, next_minterm_table

In [19]:
def simplify_prime_implicants(group_table, minterm_table):
    uninv_bits_counts = list(group_table.keys())
    next_group_table = defaultdict(list)
    next_minterm_table = {}
    unique_bits = set()
    for idx in range(len(uninv_bits_counts)-1):
        current_group, next_group = uninv_bits_counts[idx], uninv_bits_counts[idx+1]
        for current_minterms in group_table[current_group]:
            for next_minterms in group_table[next_group]:
                count = 0
                variable = ''
                current_bits = minterm_table[current_minterms]
                next_bits = minterm_table[next_minterms]

                for byte_idx in range(0,4):
                    current_byte = current_bits[byte_idx]
                    next_byte = next_bits[byte_idx]
                    if current_byte == next_byte:
                        variable += current_bits[byte_idx]
                    else:
                        variable += '-'
                        count += 1

                if count == 1 and variable not in unique_bits:
                    next_group_table[idx].append((current_minterms+next_minterms))
                    next_minterm_table[current_minterms+next_minterms] = variable
                    unique_bits.add(variable)
                    
    return next_group_table, next_minterm_table

In [20]:
next_group_table, next_minterm_table = simplify_prime_implicants(group_table, minterm_table)

In [21]:
next_group_table

defaultdict(list,
            {0: [(0, 1), (0, 2), (0, 8)],
             1: [(1, 3), (1, 9), (2, 3), (2, 6), (8, 9)],
             2: [(3, 7), (6, 7), (6, 14)],
             3: [(7, 15), (14, 15)]})

In [22]:
next_minterm_table

{(0, 1): '000-',
 (0, 2): '00-0',
 (0, 8): '-000',
 (1, 3): '00-1',
 (1, 9): '-001',
 (2, 3): '001-',
 (2, 6): '0-10',
 (8, 9): '100-',
 (3, 7): '0-11',
 (6, 7): '011-',
 (6, 14): '-110',
 (7, 15): '-111',
 (14, 15): '111-'}

In [23]:
next_group_table2, next_minterm_table2 = simplify_prime_implicants(next_group_table, next_minterm_table)

In [24]:
next_group_table2

defaultdict(list,
            {0: [(0, 1, 2, 3), (0, 1, 8, 9)],
             1: [(2, 3, 6, 7)],
             2: [(6, 7, 14, 15)]})

In [25]:
next_minterm_table2

{(0, 1, 2, 3): '00--',
 (0, 1, 8, 9): '-00-',
 (2, 3, 6, 7): '0-1-',
 (6, 7, 14, 15): '-11-'}

In [35]:
max_bit = format(max_int, 'b')
most_significant_bit = len(max_bit)
minterm_matrix = np.zeros((most_significant_bit,len(integers)), dtype='int')
print(minterm_matrix)

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [36]:
integers.index(7)

5

In [37]:
for idx, minterm in enumerate(next_minterm_table2.keys()):
    row = minterm_matrix[idx, :]
    for bit in minterm:
        row_idx = integers.index(bit)
        row[row_idx] = 1
    

In [38]:
integers

[0, 1, 2, 3, 6, 7, 8, 9, 14, 15]

In [39]:
next_minterm_table2.keys()

dict_keys([(0, 1, 2, 3), (0, 1, 8, 9), (2, 3, 6, 7), (6, 7, 14, 15)])

In [66]:
minterm_matrix

array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 1, 1]])

In [67]:
col_sums = np.sum(minterm_matrix, axis = 0)
min_col_sums = min(col_sums)
print(min_col_sums)
indicies = np.where(col_sums == min_col_sums)[0]
print(indicies)

1
[6 7 8 9]


In [68]:
next_matrix = np.delete(minterm_matrix, indicies, axis = 1)
print(next_matrix)

[[1 1 1 1 0 0]
 [1 1 0 0 0 0]
 [0 0 1 1 1 1]
 [0 0 0 0 1 1]]


In [69]:
integers

[0, 1, 2, 3, 6, 7, 8, 9, 14, 15]

In [73]:
next_prime_implicants = np.delete(integers, indicies, axis = 0)
print(next_prime_implicants)

[0 1 2 3 6 7]


10

In [50]:
test = minterm_matrix[: , -1]
print(test)
idx = np.where(test == 1)[0][0]
print(idx)

[0 0 0 1]
3


In [31]:
next_group_table3, next_minterm_table3 = simplify_prime_implicants(next_group_table2, next_minterm_table2)

In [32]:
next_group_table3

defaultdict(list, {})

In [33]:
next_minterm_table3

{}

In [143]:
unique_check = set()
for minterms in next_minterm_table2.keys():
    for group in minterms:
        if group not in unique_check:
            unique_check.add(group)
        else:
            unique_check.remove(group)
print(unique_check)

{8, 9, 14, 15}


In [147]:
keys_unique_groups = []
vals_unique_groups = []
for key in next_minterm_table2:
    if any(group in unique_check for group in key):
        keys_unique_groups.append(key)
        vals_unique_groups.append(next_minterm_table2[key])
print(keys_unique_groups)
print(vals_unique_groups)

[(0, 1, 8, 9), (6, 7, 14, 15)]
['-00-', '-11-']


In [150]:
results = []
for minterms, bits in next_minterm_table2.items():
    final_result = ''
    if minterms not in keys_unique_groups:
        for unique_key in keys_unique_groups:
            final_result += next_minterm_table2[unique_key] + '+'
        final_result += bits
        results.append(final_result)

In [151]:
print(results)

['-00-+-11-+00--', '-00-+-11-+0-1-']
